---
# --- Day 13: Shuttle Search ---
---

In [18]:
import numpy as np

from sympy import Matrix, zeros
from diophantine import lllhermite, get_solutions
from itertools import chain

### Input

In [2]:
with open("data/13_input.txt") as f:
    data = [l.strip() for l in f.readlines()]

In [3]:
arrival_ts = int(data[0])
bus_ids = [int(i) for i in data[1].split(",") if i != "x"]

### Part 1: earliest bus

In [4]:
wait_times = [int(np.ceil(arrival_ts/i))*i-arrival_ts for i in bus_ids]

In [5]:
imin = np.argmin(wait_times)
wt = wait_times[imin]
bid = bus_ids[imin]
print(f"Earliest bus is #{bid} and we have to wait {wt} minutes. Multiplication=<{bid*wt}>.")

Earliest bus is #797 and we have to wait 6 minutes. Multiplication=<4782>.


### Part 2: earliest timestamp that fits

In [13]:
test_string = "17,x,13,19"

#### Smartest way

In [7]:
def get_ids_shifts(bus_string):
    bus_ids = np.array(bus_string.split(","))
    shifts = np.array(range(0, len(bus_ids)))
    nox = bus_ids != "x"
    bus_ids = bus_ids[nox].astype('int')
    shifts = shifts[nox]
    return bus_ids, shifts

In [8]:
def check_timestamp(num, bus_ids, shifts, verbose=False):
    ok = True
    j = 0
    while ok and (j < len(bus_ids)):
        bid = bus_ids[j]
        shifted_ts = num + shifts[j]
        if verbose:
            print(f"Bus ID <{bid}>...", end=" ")
        ok = (shifted_ts%bid) == 0
        if ok:
            if verbose:
                print(f"OK ({int(shifted_ts/bid)} times).")
            j += 1
        else:
            j += 1
            if verbose:
                print("NO!")
    return ok

In [9]:
def find_smallest_positive_solution(sol, k):
    if sol <= 0:
        while sol <=0:
            sol += k
    else:
        while sol > 0:
            sol -= k
        sol += k
    return sol

In [10]:
def get_solution_basis(A, b):
    A = Matrix(A)
    b = Matrix(b)
    if b.shape != (A.shape[0], 1):
        raise Exception("Length of b vector ({}) does not match number of rows"
                        " in A matrix ({})".format(b.shape[0], A.shape[0]))
    G = zeros(A.shape[1] + 1, A.shape[0] + 1)
    G[:-1, :-1] = A.T
    G[-1, :-1] = b.reshape(1, b.shape[0])
    G[-1, -1] = 1
    # A is m x n, b is m x 1, solving AX=b, X is n x 1+
    # Ab is the (n+1) x m transposed augmented matrix. G=[A^t|0] [b^t]1]
    hnf, P, rank = lllhermite(G)
    r = rank - 1  # For convenience
    if not any(chain(hnf[:r, -1], hnf[r, :-1])) and hnf[r, -1] == 1:
        nullity = hnf.shape[0] - rank
        if nullity:
            basis = P[rank:, :-1].col_join(-P[r, :-1])
            solutions = get_solutions(basis)
        else:
            raise NotImplementedError("Ax=B has unique solution in integers")
    else:
        solutions = []
    result_ok = A*solutions[0] == b
    k = int(basis[:,0][0])*np.sign(basis[:,0][0])
    sol = find_smallest_positive_solution(int(solutions[0][0]), k)
    
    return sol, k, result_ok

In [16]:
def find_earliest_timestamp_by_splitting(bids, shifts, n_cut=6, max_iterations=1e6):
    if n_cut < len(bids):
        bids_solver = bids[:n_cut] 
        shifts_solver = shifts[:n_cut]
        bids_manu = bids[n_cut:]
        shifts_manu = shifts[n_cut:]   
        n = len(bids_solver)
        A = np.hstack([np.ones((n,1))*(-1), np.diag(bids_solver)])
        b = shifts_solver[:]
        sol, k, result_ok = get_solution_basis(A, b)
        if result_ok:
            i = 1
            found = check_timestamp(sol, bids_manu, shifts_manu)
            while not found and (i <= max_iterations):
                sol += k
                found = check_timestamp(sol, bids_manu, shifts_manu)
                i += 1
            if found:
                found_ts = sol
                print(f"Earliest timestamp found: <{found_ts}>.")
            else:
                found_ts = None
                print("I am sorry, no match found in the considered window.")
        else:
            found_ts = None
            print("I am sorry, no good solution has been found, try to reduce the initial dimension.")
    else:
        n = len(bids)
        A = np.hstack([np.ones((n,1))*(-1), np.diag(bids)])
        b = shifts[:]
        sol, k, result_ok = get_solution_basis(A, b)
        if result_ok:
            found_ts = sol
            print(f"Earliest timestamp found: <{found_ts}>.")
        else:
            found_ts = None
            print("I am sorry, no good solution has been found, try to reduce the initial dimension.")
    return found_ts

In [19]:
bus_ids, shifts = get_ids_shifts(test_string)
ts = find_earliest_timestamp_by_splitting(bus_ids, shifts)
if ts is not None:
    check_timestamp(ts, bus_ids, shifts, verbose=True)

Earliest timestamp found: <3417>.
Bus ID <17>... OK (201 times).
Bus ID <13>... OK (263 times).
Bus ID <19>... OK (180 times).


In [20]:
bus_ids, shifts = get_ids_shifts(data[1])
ts = find_earliest_timestamp_by_splitting(bus_ids, shifts)
if ts is not None:
    check_timestamp(ts, bus_ids, shifts, verbose=True)

Earliest timestamp found: <1118684865113056>.
Bus ID <19>... OK (58878150795424 times).
Bus ID <37>... OK (30234726084137 times).
Bus ID <883>... OK (1266913777025 times).
Bus ID <23>... OK (48638472396221 times).
Bus ID <13>... OK (86052681931776 times).
Bus ID <17>... OK (65804992065476 times).
Bus ID <797>... OK (1403619655098 times).
Bus ID <41>... OK (27284996710076 times).
Bus ID <29>... OK (38575340176315 times).


#### [Not working] Really brute force

In [21]:
def transform_bus_list(bus_string):
    bus_ids = np.array(bus_string.split(","))
    shifts = np.array(range(0, len(bus_ids)))
    nox = bus_ids != "x"
    bus_ids = bus_ids[nox].astype('int')
    shifts = shifts[nox]
    # let's start with the largest bus id
    imax = np.argmax(bus_ids)
    bus_ids = np.hstack([bus_ids[imax:], bus_ids[:imax]])    
    delta = shifts[imax]
    shifts = np.hstack([shifts[imax:], shifts[:imax]]) - delta
    return bus_ids, shifts, delta

In [22]:
def find_earliest_timestamp_bf(bus_string, num_start=0, num_stop=1e6, verbose_step=1e7):
    bids, shifts, delta = transform_bus_list(bus_string)
    i = int(np.floor(num_start/bids[0])) + 1
    found = False
    limit_passed = False
    cnt = 1
    while not (found or limit_passed):
        n = bids[0]*i
        limit_passed = n >= num_stop
        ts = n - delta
        if (cnt%verbose_step==0):
            print(f"Iteration <{cnt}>, trying timestamp <{ts}>...")
        j = 1
        fit = True
        while fit and (j < len(bids)):
            fit = (n + shifts[j]) % bids[j] == 0
            j += 1
        if fit == True:
            found = True
        else:
            i += 1
            cnt += 1
    if found:
        found_ts = ts
        print(f"Earliest timestamp found: <{found_ts}>.")
    else:
        found_ts = None
        print("I am sorry, no match found in the considered window.")
    return found_ts

In [24]:
ts = find_earliest_timestamp_bf(test_string)

Earliest timestamp found: <3417>.


In [25]:
find_earliest_timestamp_bf(data[1], num_start=528398731372477, num_stop=7*10e14)

KeyboardInterrupt: 